In [111]:
%matplotlib inline

import arcpy
import os, shutil
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

workingDir = "Z:\Auxiliary\Project_Folders\CS_Sandbox\\NCALM\\bulking\\scratch"
arcpy.env.workspace=workingDir
arcpy.env.overwriteOutput = 1

In [112]:
# Assign datasets
# Debris flow polygons
dfpoly = 'Full_DF_Vector\df_polys_trunk'
# Debris flow initiation points
dfpoint = 'Full_DF_Vector\df_initiation_points'
# Lidar DEM
ncalm_el = 'NCALM_version2\ncalm_el_mosaic.tif' # need to mosaic grg and prv elevation rasters
# Change detection DEM (DoD or cloud based difference image)
cdiRas = 'NCALM_version2\cdi_1stdev.tif' # need to mosaic grg and prv CDI rasters
# Flow direction raster
fdir = 'Hydrology\\fdir.tif'
# Flow accumulation (area not pixels) raster
facc = 'Hydrology\\facc_area.tif'

# Assign variables
dx = 10 # distance down the track to place points

# Set up directories
tempDir = os.path.join(workingDir, 'temp')
try:
    os.mkdir(tempDir)
    print('Temporary output directory created')
except:
    print('WARNING: Temporary directory already exists, overwriting...')
    
outDir = os.path.join(workingDir, 'outputs')
try:
    os.mkdir(outDir)
    print('Output directory created')
except:
    print('WARNING: Output directory already exists, overwriting...')
    
indDir = os.path.join(outDir, 'individual')
try:
    os.mkdir(indDir)
    print('Individual output directory created')
except:
    print('WARNING: Individual output directory already exists, overwriting...')

In [113]:
# Create cost paths from initiation points
cpRas = arcpy.sa.CostPath(dfpoint, facc, fdir, "EACH_CELL", "cp_code")
print('Cost paths computed')

# Cost paths to line
df_lines = os.path.join(outDir, 'df_lines.shp')
arcpy.RasterToPolyline_conversion(cpRas, df_lines, simplify="SIMPLIFY")

Cost paths computed


<Result 'Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\scratch\\outputs\\df_lines.shp'>

###############################
## Check lines in map view - if flow lines exit debris flows, modify polygon
## If lines look good:
1. Copy Feature Class to "df_lines_ind.shp"
2. Manually select/merge lines from each initiation point - captures coalescing flows
3. Add "df_id" column
4. Add spatial join from df_lines_ind to df_points
5. Populate df_lines_ind.shp [df_id] from the spatial join with df_points [init_id]
6. Remove small off-shoots from main flowlines (artifacts from the cost path processing)
7. Smooth lines with smoothing factor ~25m

###############################

In [1]:
# Iterate through debris flow polygons and process each - dissolve lines in each polygon into one line and clip to polygon

fields = ['df_id']
arcpy.MakeFeatureLayer_management('df_polys','dfpoly_select')

with arcpy.da.SearchCursor('dfpoly_select', fields) as sc:
    for c, row in enumerate(sc):
        print('Starting debris flow {0}'.format(c))
        dfl='df{0}_lines'.format(c)
        where = 'df_id = {0}'.format(row[0])
        arcpy.MakeFeatureLayer_management('df_lines',dfl)
        arcpy.SelectLayerByAttribute_management('dfpoly_select','NEW_SELECTION', where)
        arcpy.SelectLayerByLocation_management(dfl, 'INTERSECT', 'dfpoly_select', selection_type="NEW_SELECTION")
        df_lineIDt = os.path.join(tempDir, 'df{0}_line_temp.shp'.format(c))
        arcpy.Dissolve_management(dfl, df_lineIDt)
        df_lineID = os.path.join(indDir, 'df{0}_line.shp'.format(c))
        arcpy.Clip_analysis(df_lineIDt, 'dfpoly_select', df_lineID)    
        

In [114]:
# Generate points along line, get distance
shps = []
for file in os.listdir(indDir):
    if file.endswith(".shp"):
        shps.append(file)
        
        
for r in shps:
    c = r[:-9][2:]
    print('Starting debris flow ID: {0}'.format(c))
    
    # Assign points along lines
    dfl = os.path.join(indDir, r)
    dfdx = os.path.join(indDir, 'df{0}_points.shp'.format(c))
    arcpy.GeneratePointsAlongLines_management(dfl, dfdx, 'DISTANCE', Distance='{0} meters'.format(dx), Include_End_Points="END_POINTS")

    # Add point ID field to shapefile
    arcpy.AddField_management(dfdx, "pid", "short")
    
    # Assign unique id to each point
    with arcpy.da.UpdateCursor(dfdx, ['Id', 'pid']) as uc:
        for cc, row in enumerate(uc):
            row[0] = int(c)
            row[1] = int(cc)
